In [1]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
import time
import pandas as pd
import numpy as np

In [2]:
sleep_time = np.random.randint(1,10)
sleep_time

4

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
def get_jobs(keyword, num_jobs, verbose):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')
    
    #Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path="/Applications/chromedriver", options=options)
    driver.set_window_size(1120, 1000)
    #url = 'https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=%22'+keyword + '%22&sc.keyword=%22data+analyst%22&locT=C&locId=1147401&jobType='
    url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword={}&locT=C&locId=1147401&locKeyword=San%20Francisco,%20CA&jobType=all&fromAge=-1&minSalary=60000&includeNoSalaryJobs=true&radius=100&cityId=-1&minRating=0.0&industryId=-1&sgocId=-1&seniorityType=all&companyId=-1&employerSizes=0&applicationType=0&remoteWorkType=0'.format(keyword)
    print(url)
    driver.get(url)
    jobs = []

    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.

        #Let the page load. Change this number based on your internet speed.
        #Or, wait until the webpage is loaded, instead of hardcoding it.
        print(jobs)
        time.sleep(11)

        #Test for the "Sign Up" prompt and get rid of it.
        try:
            driver.find_element_by_class_name("selected").click()
        except ElementClickInterceptedException:
            pass

        time.sleep(.1)

        try:
            driver.find_element_by_css_selector('[alt = "Close"]').click()
            
        except NoSuchElementException:
            pass
        
        
        #/html/body/header/nav[1]/div/div/div/div[4]/div[3]/form/div/button

        try:
            #driver.find_element_by_name('search')
            #driver.find_elemnt_by_css_selector("button")
            x_path = '//*[@id="scBar"]/div/button/span'
            driver.find_element_by_xpath(x_path).click()

            #//*[@id="scBar"]/div/button/span
            #<button class="gd-ui-button ml-std col-auto css-iixdfr" type="submit" data-test="search-bar-submit"><span>Search</span></button>
            #//*[@id="scBar"]/div/button
            print('hit search button')
        except:
            print('could not hit search button')
            pass
        
        #<a class="login-btn" href="javascript:;" data-bind="click:loginSection.loginClick">
   # <span class="btn-text">Login</span>
#</a>
        #driver.find_element_by_css_selector("a.login-btn > span.btn-text").click()
        
        
        
        #Going through each job in this page
        job_buttons = driver.find_elements_by_class_name("jl")  #jl for Job Listing. These are the buttons we're going to click.
        for job_button in job_buttons:  

            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            
            sleep_time = np.random.randint(1,10)
            time.sleep(sleep_time)
            if len(jobs) >= num_jobs:
                break
            try:
                job_button.click()#You might 
            except:
                continue
                
            
            time.sleep(1)
            collected_successfully = False

            while not collected_successfully:
                try:
                    company_name = driver.find_element_by_xpath('.//div[@class="employerName"]').text
                    location = driver.find_element_by_xpath('.//div[@class="location"]').text
                    job_title = driver.find_element_by_xpath('.//div[contains(@class, "title")]').text
                    job_description = driver.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text
                    collected_successfully = True
                except:
                    time.sleep(5)

            try:
                salary_estimate = driver.find_element_by_xpath('.//span[@class="gray salary"]').text
                
            except NoSuchElementException:
                salary_estimate = -1 #You need to set a "not found value. It's important."
            
            try:
                rating = driver.find_element_by_xpath('.//span[@class="rating"]').text
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."

            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            #Going to the Company tab...
            #clicking on this:
            #<div class="tab" data-tab-type="overview"><span>Company</span></div>
            try:
                driver.find_element_by_xpath('.//div[@class="tab" and @data-tab-type="overview"]').click()

                try:
                    #<div class="infoEntity">
                    #    <label>Headquarters</label>
                    #    <span class="value">San Francisco, CA</span>
                    #</div>
                    headquarters = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Headquarters"]//following-sibling::*').text
                except NoSuchElementException:
                    headquarters = -1

                try:
                    size = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Size"]//following-sibling::*').text
                except NoSuchElementException:
                    size = -1

                try:
                    founded = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Founded"]//following-sibling::*').text
                except NoSuchElementException:
                    founded = -1

                try:
                    type_of_ownership = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Type"]//following-sibling::*').text
                except NoSuchElementException:
                    type_of_ownership = -1

                try:
                    industry = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Industry"]//following-sibling::*').text
                except NoSuchElementException:
                    industry = -1

                try:
                    sector = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Sector"]//following-sibling::*').text
                except NoSuchElementException:
                    sector = -1

                try:
                    revenue = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Revenue"]//following-sibling::*').text
                except NoSuchElementException:
                    revenue = -1

                try:
                    competitors = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Competitors"]//following-sibling::*').text
                except NoSuchElementException:
                    competitors = -1

            except:  #Rarely, some job postings do not have the "Company" tab.
                headquarters = -1
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1
                competitors = -1

                
            #if verbose:
            print("job Title: {}".format(job_title))
#             print("Headquarters: {}".format(headquarters))
#             print("Size: {}".format(size))
#             print("Founded: {}".format(founded))
#             print("Type of Ownership: {}".format(type_of_ownership))
#             print("Industry: {}".format(industry))
#             print("Sector: {}".format(sector))
#             print("Revenue: {}".format(revenue))
#             print("Competitors: {}".format(competitors))
#             print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({"Job Title" : job_title,
            "Salary Estimate" : salary_estimate,
            "Job Description" : job_description,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location,
            "Headquarters" : headquarters,
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue,
            "Competitors" : competitors})
            
            #add job to jobs

        #Clicking on the "next page" button
        try:
            driver.find_element_by_xpath('.//li[@class="next"]//a').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break

    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.

In [5]:
#This line will open a new chrome window and start the scraping.
df = get_jobs("data analyst", 30, False)
df2 = get_jobs("project manager", 20, False)
df3 = get_jobs("product manager", 20, False)
df4 = pd.concat([df,df2], ignore_index = True)

df4.to_csv("jobs_df1.csv")

https://www.glassdoor.com/Job/jobs.htm?sc.keyword=data analyst&locT=C&locId=1147401&locKeyword=San%20Francisco,%20CA&jobType=all&fromAge=-1&minSalary=60000&includeNoSalaryJobs=true&radius=100&cityId=-1&minRating=0.0&industryId=-1&sgocId=-1&seniorityType=all&companyId=-1&employerSizes=0&applicationType=0&remoteWorkType=0
[]


AttributeError: 'WebDriver' object has no attribute 'find_element_by_class_name'

In [ ]:
df.to_csv('data_analyst_scrape.csv')

In [ ]:
df2 = get_jobs("product manager", 30, False)
df2.to_csv('product_manager_scrape.csv')
df2

In [ ]:
df3 = get_jobs("project manager", 45, False)

df3.to_csv('project_manager_scrape.csv')

In [ ]:
 df4 = get_jobs("marketing analyst specialist", 45, False)

In [ ]:
 df4.to_csv('marketing_scrape.csv')
    df4

In [ ]:
 df4.to_csv('marketing_analyst_scrape.csv')

In [ ]:
df5 = get_jobs("account manager", 45, False)
df5.to_csv('account_manager_scrape2.csv')

In [ ]:
# df6 = get_jobs("ux designer", 45, False)
# df6.to_csv('ux_designer_scrape.csv')

In [ ]:
#df7 = get_jobs("strategy consultant", 45, False)
#df7.to_csv('consulting.csv')

In [18]:
#df7.to_csv('Scraped_Data/consulting_scrape.csv')

In [ ]:
df8 = get_jobs("data scientist", 45, False)
df8.to_csv('Scraped_Data/data_scientist_scrape.csv')

In [ ]:
x = ['senior', 'data', 'analyst', '1','2']
y = ['data', 'analyst']

x if y[0] in x else 0

In [ ]:
def check_keywords(keywords, string_list):
    keyword_list = list(keywords.split(','))
    check = True
    for word in keyword_list:
        if word not in string_list:
            print(word)
            check = False
    return check, keyword_list, string_list

check_keywords('data,analyst', x)